<img src="img/noback.png" height=20% width=20%>

# Gassman Fluid Substitution
Predict the changes of P Wave velocity, when one fluid is replaced with another.  
Copyright (C) 2020 ezygeo.com, Hadyan Pratama (hadyan_pratama@rocketmail.com)



## Import Library 

In [29]:
import numpy as np

## Define Formula 

#### 1. K Saturated Initial

_Biot (1956)_

<img src="img/biot.png" height=50% width=50%>

In [30]:
def msat(vs, rho):
    '''
    Miu saturated (C) hpratama 2020
    
    Input:
    vs = Shear Velocity (Km/s)
    rho = Density (g/cc)
  
    '''
    return rho*(vs**2)

In [31]:
def ksat1(vp, vs, rho):
    '''
    K saturated (C) hpratama 2020
    
    Input:
    vp = P wave velocity (Km/s)
    vs = Shear wave Velocity (Km/s)
    rho = Density (g/cc)
  
    '''
    Ksat1 = ((vp**2)*rho) - 4/3*(msat(vs, rho))
    return np.round(Ksat1, 3)

#### 2. K Fluid 

_Batzle and Wang, 1992_

<img src="img/batzleandwang.png" height=40% width=40%>

In [32]:
def kfl(sw, khc, kw):
    '''
    K Fluid (C) hpratama 2020
    
    Input:
    sw = Water Saturation
    khc = K Hydrocarbon
    kw = K Water
    
    '''
    kfl = ((sw/kw) + ((1-sw)/khc))**-1
    return np.round(kfl, 3)

#### 3. K Mineral

_Voight-Reuss-Hill Average_

<img src="img/voightreusshill.png" height=50% width=50%>

In [47]:
def km(por, fq, fc, kq, kc):
    '''
    K mineral (K0) (C) hpratama 2020
    
    Input:
    por = Porosity
    fq = Quartz Fraction
    fc = Calcite Fraction
    kq = Quarzt Bulk
    kc = Calcite Bulk
    
    '''
    kv = ((fq)*kq) + ((fc)*kc) 
    
    kr = (((fq)/kq) + ((fc)/kc))**(-1)
    
    km = (kv + kr)/2
    
    return np.round(km, 3)


#### 4. K Saturated Final 

_Mavko et al._

<img src="img/mavko.png" height=50% width=50%>

In [48]:
def ksat2(ksat1, k0, kfl1, kfl2, por):
    '''
    K Saturation after fluid substitution (C) hpratama 2020
    
    Input:
    ksat1 = K Saturation before Fluid Substitution
    k0 = K Mineral
    kfl1 = K Fluid before Substitution
    kfl2 = K Fluid after Substitution
    por = Porosity
    
    '''
    a = (ksat1/(k0 - ksat1)) - (kfl1/(por*(k0 - kfl1))) + (kfl2/(por*(k0 - kfl2)))
    ksat2 = (a*(k0))/(1+a)
    return np.round(ksat2, 3)

#### 5. Vp 

<img src="img/rho_2.png" height=50% width=50%>

In [49]:
def rhofluid(sw, so, rhoo, rhow):
    
    rho1 = ((1-so) * rhow) + (so * rhoo)
    rho2 = ((sw) * rhow) + ((1 - sw) * rhoo)
    rho1 = np.round(rho1, 3)
    rho2 = np.round(rho2, 3)    
    return rho1, rho2

In [50]:
def rhosat(rho, por, rhow, sw, so, rhoo):
    rho1, rho2 = rhofluid(sw, so, rhoo, rhow)
    
    rhosat = rho+(por*(rho2 - rho1))
    return np.round(rhosat, 3)

In [51]:
def vp(ksat2, vs, rhosat):
    vp = ((ksat2 + (4/3*(msat(vs, rhosat)))) / (rhosat))**0.5
    return np.round(vp, 3)

## Data

In [52]:
Vp = 3.349
Vs = 1.836

Sw = 1
So = 0.7

Por = 0.3

Fq = 0.7
Fc = 0.3
Kq = 36
Kc = 75
Ko = 1.6
Kw = 2.83

Rho = 2.13
Rhoo = 0.8
Rhow = 1

## Calculate 

To get new value of Vp, there is some variable that need to calculate.

* First we need to calculate K saturation before fluid subtitution. This K saturartion could be calculated from the old 

In [53]:
Ksat1 = ksat1(Vp, Vs, Rho)

* Then, we estimate the K fluid before and after fluid substitution

In [54]:
Kfl1 = kfl((1-So), Ko, Kw)
Kfl2 = kfl(Sw, Ko, Kw)

* After that, we evaluate K Mineral that can be estimate from Hill formula, this formula need Voight-Reuss Bounds calculation too. First we need to calculate K saturation before fluid subtitution.

In [55]:
K0 = km(Por, Fq, Fc, Kq, Kc)
print(K0)

45.177


* Here we calculate K saturated after fluid substitution

In [56]:
Ksat2 = ksat2(Ksat1, K0, Kfl1, Kfl2, 0.3)

* To calculate the new Vp, we need to calculate density saturation. After that we estimate new Vp with new K saturated after fluid substitution

In [57]:
Rhosat = rhosat(Rho, Por, Rhow, Sw, So, Rhoo)

In [58]:
Vp = vp(Ksat2, Vs, Rhosat)

In [59]:
print(Vp)

3.44


Done